The monkeys take you on a surprisingly easy trail through the jungle. They're even going in roughly the right direction according to your handheld device's Grove Positioning System.

As you walk, the monkeys explain that the grove is protected by a force field. To pass through the force field, you have to enter a password; doing so involves tracing a specific path on a strangely-shaped board.

At least, you're pretty sure that's what you have to do; the elephants aren't exactly fluent in monkey.

The monkeys give you notes that they took when they last saw the password entered (your puzzle input).

For example:

```
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
```

The first half of the monkeys' notes is a map of the board. It is comprised of a set of open tiles (on which you can move, drawn `.`) and solid walls (tiles which you cannot enter, drawn `#`).

The second half is a description of the path you must follow. It consists of alternating numbers and letters:

A number indicates the number of tiles to move in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.
A letter indicates whether to turn 90 degrees clockwise (`R`) or counterclockwise (`L`). Turning happens in-place; it does not change your current tile.
So, a path like `10R5` means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".

You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing to the right (from the perspective of how the map is drawn).

If a movement instruction would take you off of the map, you wrap around to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.

For example, if you are at A and facing to the right, the tile in front of you is marked B; if you are at C and facing down, the tile in front of you is marked D:

```
        ...#
        .#..
        #...
        ....
...#.D.....#
........#...
B.#....#...A
.....C....#.
        ...#....
        .....#..
        .#......
        ......#.
```

It is possible for the next tile (after wrapping around) to be a wall; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.

By drawing the last facing you had with an arrow on each tile you visit, the full path taken by the above example looks like this:

```
        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
>>>v...>#.>>    
..#v...#....    
...>>>>v..#.    
        ...#....
        .....#..
        .#......
        ......#.
```
To finish providing the password to this strange input device, you need to determine numbers for your final row, column, and facing as your final position appears from the perspective of the original map. Rows start from 1 at the top and count downward; columns start from 1 at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is 0 for right (`>`), 1 for down (`v`), 2 for left (`<`), and 3 for up (`^`). The final password is the sum of 1000 times the row, 4 times the column, and the facing.

In the above example, the final row is 6, the final column is 8, and the final facing is 0. So, the final password is `1000 * 6 + 4 * 8 + 0: 6032`.

Follow the path given in the monkeys' notes. ***What is the final password?***

In [84]:
import numpy as np
import re
input_file  = open("data/2022-12-22-input.txt")
input_lines = input_file.readlines()
monkey_map = np.array([list(line[:-1].ljust(150, ' ')) for line in input_lines if len(line) > 5 and not line[0].isdigit()])
print(monkey_map.shape)
directions = input_lines[-1].strip()
distances = [int(i) for i in re.split('[LR]', directions)]
rotations = re.split('[0-9]+', directions)[1:-1]
print(f'{len(distances)} distances, {len(rotations)} rotations')

(200, 150)
2001 distances, 2000 rotations


In [86]:
map_strings = [
    "        ...#",
    "        .#..",
    "        #...",
    "        ....",
    "...#.......#",
    "........#...",
    "..#....#....",
    "..........#.",
    "        ...#....",
    "        .....#..",
    "        .#......",
    "        ......#."
]
test_monkey_map = np.array([list(line.ljust(20, ' ')) for line in map_strings])
test_directions = '10R5L5R10L4R5L5'
test_distances = [int(i) for i in re.split('[LR]', test_directions)]
test_rotations = re.split('[0-9]+', test_directions)[1:-1]

In [87]:
map_height, map_width = monkey_map.shape
start_location = np.array([0, np.argmax(monkey_map[0, :] == '.')])
movements = [[0, 1], [1, 0], [0, -1], [-1, 0]]
movement_index = 0

def StraightLineMove(monkey_map, current_location, movement, steps):
    if steps == 0: return current_location
    next_location = current_location + movement
    next_location[0] = next_location[0] % monkey_map.shape[0]
    next_location[1] = next_location[1] % monkey_map.shape[1]
    while monkey_map[tuple(next_location)] == ' ':
        next_location = next_location + movement
        next_location[0] = next_location[0] % monkey_map.shape[0]
        next_location[1] = next_location[1] % monkey_map.shape[1]        
    match monkey_map[tuple(next_location)]:
        case '.':
            return StraightLineMove(monkey_map, next_location, movement, steps - 1)
        case '#':
            return current_location

my_location = start_location
for i in range(len(distances)):
    my_location = StraightLineMove(monkey_map, my_location, movements[movement_index], distances[i])
    if i < len(rotations):
        movement_index = (movement_index + (1 if rotations[i] == 'R' else -1)) % 4

print(my_location)
print(f'The final password is {(my_location[0] + 1) * 1000 + (my_location[1] + 1) * 4 + movement_index}')
# 74384 too low

[74 96]
The final password is 75388


As you reach the force field, you think you hear some Elves in the distance. Perhaps they've already arrived?

You approach the strange input device, but it isn't quite what the monkeys drew in their notes. Instead, you are met with a large cube; each of its six faces is a square of 50x50 tiles.

To be fair, the monkeys' map does have six 50x50 regions on it. If you were to carefully fold the map, you should be able to shape it into a cube!

In the example above, the six (smaller, 4x4) faces of the cube are:

```
        1111
        1111
        1111
        1111
222233334444
222233334444
222233334444
222233334444
        55556666
        55556666
        55556666
        55556666
```

You still start in the same position and with the same facing as before, but the wrapping rules are different. Now, if you would walk off the board, you instead proceed around the cube. From the perspective of the map, this can look a little strange. In the above example, if you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up:

```
        ...#
        .#..
        #...
        ....
...#.......#
........#..A
..#....#....
.D........#.
        ...#..B.
        .....#..
        .#......
        ..C...#.
```

Walls still block your path, even if they are on a different face of the cube. If you are at E facing up, your movement is blocked by the wall marked by the arrow:

```
        ...#
        .#..
     -->#...
        ....
...#..E....#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.
```

Using the same method of drawing the last facing you had with an arrow on each tile you visit, the full path taken by the above example now looks like this:


```
        >>v#    
        .#v.    
        #.v.    
        ..v.    
...#..^...v#    
.>>>>>^.#.>>    
.^#....#....    
.^........#.    
        ...#..v.
        .....#v.
        .#v<<<<.
        ..v...#.
```

The final password is still calculated from your final position and facing from the perspective of the map. In this example, the final row is 5, the final column is 7, and the final facing is 3, so the final password is 1000 * 5 + 4 * 7 + 3 = 5031.

Fold the map into a cube, then follow the path given in the monkeys' notes. ***What is the final password?***

In [89]:
face1 = monkey_map[0:50, 50:100]
face2 = monkey_map[0:50, 100:150]
face3 = monkey_map[50:100, 50:100]
face4 = monkey_map[100:150, 0:50]
face5 = monkey_map[100:150, 50:100]
face6 = monkey_map[150:200, 0:50]
faces = [face1, face2, face3, face4, face5, face6]
#   0 1
#   2
# 3 4
# 5
# [R D L U]
next_faces0 = [1, 2, 3, 5]
next_faces1 = [4, 2, 0, 5]
next_faces2 = [1, 4, 3, 0]
next_faces3 = [4, 0, 5, 2]
next_faces4 = [1, 5, 3, 2]
next_faces5 = [4, 3, 0, 1]
rotation_01 = [[1, 0], [0, 1]]
rotation_02 = [[1, 0], [0, 1]]

(50, 50)